In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mlflow
import dagshub
import os
import pathlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
PATH = pathlib.Path('facial-emotion-recognition-augmented')
LABELS = os.listdir(PATH)

In [ ]:
dagshub.init(repo_owner='IdjiotSandwiches', repo_name='face-emotion-recognition', mlflow=True)

In [ ]:
K_SIZE = (31,31)
SIGMA = 4
THETA_RANGE = np.arange(0, np.pi, np.pi/128)
LAMBD = 10.0
GAMMA = 0.5
PSI = 0

gabor_params = {
    'ksize': K_SIZE,
    'sigma': SIGMA,
    'lambd': LAMBD,
    'gamma': GAMMA,
    'psi': PSI
}

In [ ]:
TEST_SIZE = 0.2
RANDOM_STATE = 42

split_params = {
    'test_size': TEST_SIZE,
    'random_state': RANDOM_STATE
}

X_train, X_test, y_train, y_test = train_test_split(images, labels, **split_params)

In [ ]:
images = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\images.npy')
labels = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\labels.npy')

In [ ]:
C = 1.0
KERNEL = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
DEGREE = [3,4]
GAMMA = ['scale', 'auto']

for kernel in KERNEL:
    for degree in DEGREE:
        for gamma in GAMMA:

            model_params = {
                'C': C,
                'kernel': kernel,
                'degree': degree,
                'gamma': gamma
            }
